<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/ThaiNovel_OCR_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell ใหม่: Mount Drive + Create Folders
from google.colab import drive
drive.mount('/content/drive')

# สร้าง folders ใน Drive
import os
BASE = '/content/drive/MyDrive/OCR'  # ← ชื่อ folder ของคุณ

for folder in ['raw_ocr', 'batches', 'cleaned_gpt', 'cleaned_claude', 'final_corpus', 'reports', 'training_pairs']:
    os.makedirs(f'{BASE}/{folder}', exist_ok=True)

print("✅ Folders ready in Drive!")

In [ ]:
# ============================================
# 📌 Block 1: Setup & Import
# ============================================
import os
import re
import json
import difflib
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Check pyperclip
try:
    import pyperclip
    CLIPBOARD_AVAILABLE = True
except ImportError:
    CLIPBOARD_AVAILABLE = False
    print("⚠️ pyperclip not installed - จะใช้ไฟล์แทน clipboard")

print("✅ Libraries loaded")

In [ ]:
# ============================================
# 📌 Block 2: Enhanced Configuration
# ============================================
class Config:
    """Configuration สำหรับ OCR Processing - Thai Novel Optimized"""

# ⚠️ แก้ paths ตรงนี้
    BASE = '/content/drive/MyDrive/OCR'  # ← folder หลักใน Drive

    # Paths
    RAW_OCR_DIR = 'raw_ocr'
    BATCHES_DIR = 'batches'
    CLEANED_GPT_DIR = 'cleaned_gpt'
    CLEANED_CLAUDE_DIR = 'cleaned_claude'
    FINAL_DIR = 'final_corpus'
    REPORTS_DIR = 'reports'
    TRAINING_PAIRS_DIR = 'training_pairs'

    # Processing parameters
    MAX_PAGES_PER_BATCH = 20
    MIN_LINE_LENGTH = 3  # บรรทัดที่สั้นกว่านี้น่าจะผิด

    # Enhanced OCR replacements for Thai novels
    OCR_REPLACEMENTS = {
        # Common OCR errors
        'เเ': 'แ',
        'ํา': 'ำ',
        'ํ า': 'ำ',
        '  ': ' ',
        '   ': ' ',
        '\t': ' ',

        # Punctuation fixes
        ' ๆ ': 'ๆ ',
        'ๆ ': 'ๆ',
        ' ๆ': 'ๆ',
        ' "': '"',
        '" ': '"',
        ' ,': ',',
        ' .': '.',

        # Common Thai novel terms
        'พวกเขๅ': 'พวกเขา',
        'ทํา': 'ทำ',
        'จๅก': 'จาก',
        'ดู่': 'ดู',
    }

    # Suspicious patterns (น่าจะเป็น OCR error)
    SUSPICIOUS_PATTERNS = [
        r'^[ก-ฮ]$',  # ตัวอักษรเดี่ยว
        r'^[a-zA-Z]$',  # ตัวอักษรภาษาอังกฤษเดี่ยว
        r'^.{1,2}$',  # คำสั้นมาก (1-2 ตัว)
        r'^\d+$',  # ตัวเลขอย่างเดียว
    ]

print("✅ Enhanced Config loaded")

In [ ]:
# ============================================
# 📌 Block 3: Novel Text Analyzer
# วิเคราะห์ลักษณะข้อความนิยาย
# ============================================
class NovelTextAnalyzer:
    """วิเคราะห์และแก้ปัญหา OCR สำหรับนิยายไทย"""

    @staticmethod
    def is_dialogue(text: str) -> bool:
        """ตรวจสอบว่าเป็นบทสนทนาหรือไม่"""
        dialogue_patterns = [
            r'^".*"',  # ขึ้นต้นและลงท้ายด้วย "
            r'".*"$',  # มี quote
            r'".*".*กล่าว',  # มีคำว่า กล่าว
            r'".*".*พูด',  # มีคำว่า พูด
            r'".*".*ตอบ',  # มีคำว่า ตอบ
            r'".*".*ถาม',  # มีคำว่า ถาม
            r'".*".*ร้อง',  # มีคำว่า ร้อง
            r'".*".*บ่น',  # มีคำว่า บ่น
        ]

        for pattern in dialogue_patterns:
            if re.search(pattern, text):
                return True
        return False

    @staticmethod
    def is_incomplete_line(text: str) -> bool:
        """ตรวจสอบว่าเป็นบรรทัดที่ไม่สมบูรณ์หรือไม่"""
        # บรรทัดที่น่าจะไม่สมบูรณ์
        if len(text) < Config.MIN_LINE_LENGTH:
            return True

        # ตรวจสอบ patterns ที่น่าสงสัย
        for pattern in Config.SUSPICIOUS_PATTERNS:
            if re.match(pattern, text.strip()):
                return True

        # ถ้าไม่มีสระเลย น่าจะผิด
        thai_vowels = 'ะาิีึืุูเแโใไ็่้๊๋ำ'
        if not any(v in text for v in thai_vowels):
            return True

        return False

    @staticmethod
    def should_merge_lines(prev_line: str, curr_line: str) -> bool:
        """ตัดสินใจว่าควรรวม 2 บรรทัดหรือไม่"""
        # ถ้าบรรทัดก่อนหน้าไม่จบประโยค
        if prev_line and not prev_line[-1] in '.!? ':
            # และบรรทัดปัจจุบันไม่ใช่บทสนทนาใหม่
            if not NovelTextAnalyzer.is_dialogue(curr_line):
                # และไม่ใช่ paragraph ใหม่ (ไม่ขึ้นต้นด้วยการเว้นวรรค)
                if not curr_line.startswith(('  ', '\t')):
                    return True
        return False

    @staticmethod
    def fix_broken_words(text: str) -> str:
        """แก้คำที่แตกหัก"""
        # Pattern สำหรับหาคำที่น่าจะแตก
        lines = text.split('\n')
        fixed_lines = []

        i = 0
        while i < len(lines):
            curr_line = lines[i].strip()

            # ถ้าเป็นบรรทัดที่น่าสงสัย
            if NovelTextAnalyzer.is_incomplete_line(curr_line):
                # ลองดูว่าควรรวมกับบรรทัดก่อนหน้าหรือถัดไปไหม
                if i > 0 and fixed_lines:
                    # ลองรวมกับบรรทัดก่อนหน้า
                    prev = fixed_lines[-1]
                    if not prev.endswith(('.', '!', '?', '"')):
                        fixed_lines[-1] = prev + curr_line
                        i += 1
                        continue

                if i < len(lines) - 1:
                    # ลองรวมกับบรรทัดถัดไป
                    next_line = lines[i + 1].strip()
                    if not NovelTextAnalyzer.is_dialogue(next_line):
                        fixed_lines.append(curr_line + next_line)
                        i += 2
                        continue

            # ถ้าไม่ต้องรวม ก็เพิ่มปกติ
            if curr_line:  # ไม่เพิ่มบรรทัดว่าง
                fixed_lines.append(curr_line)
            i += 1

        return '\n'.join(fixed_lines)

print("✅ NovelTextAnalyzer ready")

In [ ]:
# ============================================
# 📌 Block 4: Enhanced BatchPreparer
# ============================================
class EnhancedBatchPreparer:
    """Enhanced batch preparer สำหรับนิยายไทย"""

    def __init__(self, input_folder=None, output_folder=None):
        self.input_folder = Path(input_folder or Config.RAW_OCR_DIR)
        self.output_folder = Path(output_folder or Config.BATCHES_DIR)
        self.input_folder.mkdir(exist_ok=True)
        self.output_folder.mkdir(exist_ok=True)
        self.analyzer = NovelTextAnalyzer()

    def pre_clean_text(self, text: str) -> str:
        """ทำความสะอาด OCR text แบบ enhanced"""

        # Step 1: Basic replacements
        for old, new in Config.OCR_REPLACEMENTS.items():
            text = text.replace(old, new)

        # Step 2: Fix broken words
        text = self.analyzer.fix_broken_words(text)

        # Step 3: Smart paragraph detection
        text = self._smart_paragraph_split(text)

        # Step 4: Clean up spacing
        text = re.sub(r'\n{3,}', '\n\n', text)  # ลดบรรทัดว่างเกิน
        text = re.sub(r' {2,}', ' ', text)  # ลด space ซ้ำ

        return text.strip()

    def _smart_paragraph_split(self, text: str) -> str:
        """แบ่ง paragraph อย่างชาญฉลาด"""
        lines = text.split('\n')
        paragraphs = []
        current_para = []

        for i, line in enumerate(lines):
            line = line.strip()

            if not line:
                # บรรทัดว่าง = จบ paragraph
                if current_para:
                    paragraphs.append(' '.join(current_para))
                    current_para = []
                continue

            # ตรวจสอบว่าเป็นบทสนทนาใหม่หรือไม่
            if self.analyzer.is_dialogue(line):
                # ถ้ามี paragraph ก่อนหน้า ให้จบมันก่อน
                if current_para and not self.analyzer.is_dialogue(current_para[-1]):
                    paragraphs.append(' '.join(current_para))
                    current_para = [line]
                else:
                    current_para.append(line)
            else:
                # ถ้าไม่ใช่บทสนทนา
                if i > 0 and current_para:
                    # ตรวจสอบว่าควรรวมกับบรรทัดก่อนหน้าไหม
                    if self.analyzer.should_merge_lines(current_para[-1], line):
                        current_para.append(line)
                    else:
                        # เริ่ม paragraph ใหม่
                        paragraphs.append(' '.join(current_para))
                        current_para = [line]
                else:
                    current_para.append(line)

        # เพิ่ม paragraph สุดท้าย
        if current_para:
            paragraphs.append(' '.join(current_para))

        return '\n'.join(paragraphs)

    def create_enhanced_prompt(self, batch_text: str) -> str:
        """สร้าง prompt ที่ละเอียดขึ้นสำหรับนิยาย"""
        prompt = f"""กรุณาแก้ไขข้อความ OCR จากนิยายภาษาไทยต่อไปนี้

กฎการแก้ไข:
1. แก้คำผิด typo และการสะกดผิด
2. แก้คำที่ขาดหาย/แตกหัก (เช่น "จะอย่าง" "ประตู" "านชำ" ที่ควรเป็นประโยคเดียวกัน)
3. ลบตัวอักษรเดี่ยวๆ ที่ไม่มีความหมาย (เช่น ก, ป, ง, T)
4. รักษารูปแบบบทสนทนา (คำพูดในเครื่องหมาย "...")
5. จัด paragraph ให้เหมาะสม - บทสนทนาแยกบรรทัด, บรรยายรวมกันเป็น paragraph
6. คงรูปแบบ markers [PAGE_XXX] และ [END_PAGE_XXX] ไว้ทุกตัว
7. ห้ามเพิ่มเนื้อหาที่ไม่มีในต้นฉบับ

ตัวอย่างการแก้:
❌ OCR ผิด:
"จะอย่าง
ประตู
านชำ
ก
ไหน"

✅ แก้เป็น:
"[ประโยคที่สมบูรณ์ตามบริบท]"

ข้อความที่ต้องแก้:

{batch_text}

กรุณาแก้ไขแล้วคืนข้อความทั้งหมดพร้อม markers"""

        return prompt

    def create_batch(self, max_pages: int = None) -> Tuple[str, int]:
        """สร้าง batch พร้อม pre-cleaning ขั้นสูง"""
        max_pages = max_pages or Config.MAX_PAGES_PER_BATCH
        files = sorted(self.input_folder.glob("*.txt"))[:max_pages]

        if not files:
            print("❌ ไม่พบไฟล์ใน folder raw_ocr/")
            return "", 0

        batch_parts = ["[START_BATCH]"]
        stats = {'total_lines': 0, 'suspicious_lines': 0, 'merged_lines': 0}

        for i, file_path in enumerate(files, 1):
            try:
                text = file_path.read_text(encoding='utf-8')

                # นับสถิติก่อน clean
                original_lines = len(text.split('\n'))

                # Clean text
                cleaned_text = self.pre_clean_text(text)

                # นับสถิติหลัง clean
                cleaned_lines = len(cleaned_text.split('\n'))
                stats['total_lines'] += original_lines
                stats['merged_lines'] += (original_lines - cleaned_lines)

                # เพิ่ม markers
                page_marker = f"[PAGE_{i:03d}]"
                end_marker = f"[END_PAGE_{i:03d}]"
                batch_parts.append(f"\n{page_marker}\n{cleaned_text}\n{end_marker}")

            except Exception as e:
                print(f"⚠️ Error reading {file_path.name}: {e}")
                continue

        batch_parts.append("\n[END_BATCH]")
        batch_text = ''.join(batch_parts)

        # บันทึก batch
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        batch_file = self.output_folder / f"batch_{timestamp}.txt"
        batch_file.write_text(batch_text, encoding='utf-8')

        print(f"✅ สร้าง batch สำเร็จ: {batch_file.name}")
        print(f"   📄 จำนวน: {len(files)} หน้า")
        print(f"   📊 สถิติ:")
        print(f"      - บรรทัดทั้งหมด: {stats['total_lines']}")
        print(f"      - บรรทัดที่รวม: {stats['merged_lines']}")
        print(f"   💾 ขนาด: ~{len(batch_text.split())} คำ")

        return batch_text, len(files)

    def prepare_and_copy(self, max_pages: int = None):
        """เตรียม batch และ copy/save"""
        batch_text, page_count = self.create_batch(max_pages)

        if page_count == 0:
            return

        # สร้าง enhanced prompt
        prompt = self.create_enhanced_prompt(batch_text)

        # Estimate tokens
        estimated_tokens = len(prompt) // 2

        # Save or copy
        if CLIPBOARD_AVAILABLE:
            try:
                pyperclip.copy(prompt)
                print(f"\n✅ Copied to clipboard!")
            except Exception as e:
                print(f"⚠️ Cannot copy: {e}")
                self._save_prompt_to_file(prompt)
        else:
            self._save_prompt_to_file(prompt)

        print(f"📊 ประมาณ {estimated_tokens:,} tokens")
        print(f"\n📝 ขั้นตอนต่อไป:")
        print("   1. เปิด ChatGPT/Claude")
        print("   2. Paste prompt")
        print("   3. รอผลลัพธ์")
        print("   4. Copy ผลลัพธ์")
        print("   5. Run parse_results")

    def _save_prompt_to_file(self, prompt: str):
        prompt_file = self.output_folder / "latest_prompt.txt"
        prompt_file.write_text(prompt, encoding='utf-8')
        print(f"💾 บันทึก prompt ไว้ที่: {prompt_file}")

print("✅ EnhancedBatchPreparer ready")

In [ ]:
# ============================================
# 📌 Block 5: Quality Validator
# ตรวจสอบคุณภาพหลัง LLM แก้
# ============================================
class QualityValidator:
    """ตรวจสอบคุณภาพของข้อความที่ผ่าน LLM แล้ว"""

    @staticmethod
    def validate_text(original: str, cleaned: str) -> Dict:
        """ตรวจสอบคุณภาพการแก้ไข"""
        issues = []

        # 1. ตรวจสอบความยาว
        len_ratio = len(cleaned) / len(original) if len(original) > 0 else 0
        if len_ratio < 0.5:
            issues.append("⚠️ ข้อความสั้นลงมาก (อาจมีการลบเนื้อหา)")
        elif len_ratio > 1.5:
            issues.append("⚠️ ข้อความยาวขึ้นมาก (อาจมีการเพิ่มเนื้อหา)")

        # 2. ตรวจสอบบทสนทนา
        orig_quotes = len(re.findall(r'"[^"]*"', original))
        clean_quotes = len(re.findall(r'"[^"]*"', cleaned))
        if abs(orig_quotes - clean_quotes) > 2:
            issues.append(f"⚠️ จำนวนบทสนทนาต่างกันมาก ({orig_quotes} -> {clean_quotes})")

        # 3. ตรวจสอบชื่อตัวละคร (ถ้าพบในต้นฉบับ)
        character_names = re.findall(r'(โคเฮ|โชตะ|อัตสึยะ)', original)
        for name in set(character_names):
            orig_count = original.count(name)
            clean_count = cleaned.count(name)
            if clean_count < orig_count * 0.8:
                issues.append(f"⚠️ ชื่อ '{name}' หายไป ({orig_count} -> {clean_count})")

        # 4. ตรวจสอบ paragraph structure
        orig_paragraphs = len([p for p in original.split('\n\n') if p.strip()])
        clean_paragraphs = len([p for p in cleaned.split('\n') if p.strip()])

        return {
            'valid': len(issues) == 0,
            'issues': issues,
            'stats': {
                'length_ratio': len_ratio,
                'dialogue_count': clean_quotes,
                'paragraph_count': clean_paragraphs,
            }
        }

    @staticmethod
    def generate_quality_report(validations: List[Dict]) -> str:
        """สร้าง quality report"""
        report = "📊 Quality Validation Report\n"
        report += "=" * 50 + "\n\n"

        total = len(validations)
        valid = sum(1 for v in validations if v['valid'])

        report += f"✅ Valid: {valid}/{total} ({valid/total*100:.1f}%)\n"
        report += f"⚠️ Issues found: {total - valid}\n\n"

        if total - valid > 0:
            report += "Issues detail:\n"
            for i, val in enumerate(validations):
                if not val['valid']:
                    report += f"\nPage {i+1}:\n"
                    for issue in val['issues']:
                        report += f"  {issue}\n"

        return report

print("✅ QualityValidator ready")